In [106]:
import pandas as pd
import numpy as np
import jieba.analyse as analyse
import jieba

# 基于tf-idf的关键词抽取 #

In [14]:
stopwords=pd.read_csv("data/stopwords.txt",index_col=False,quoting=3,names=['stopword'], encoding='utf-8')

In [171]:
df = pd.read_csv('data/car_news.csv', encoding='utf-8')  #中文记得用utf-8

In [172]:
df = df.dropna()
lines=df.content.values.tolist()

In [104]:
content = " ".join(lines)
print(" ".join(analyse.extract_tags(content, topK=2, withWeight=False, allowPOS=())))

汽车 2016


# 基于TextRank的关键词抽取 #

In [60]:
help(analyse.textrank)

Help on method textrank in module jieba.analyse.textrank:

textrank(sentence, topK=20, withWeight=False, allowPOS=('ns', 'n', 'vn', 'v'), withFlag=False) method of jieba.analyse.textrank.TextRank instance
    Extract keywords from sentence using TextRank algorithm.
    Parameter:
        - topK: return how many top keywords. `None` for all possible words.
        - withWeight: if True, return a list of (word, weight);
                      if False, return a list of words.
        - allowPOS: the allowed POS list eg. ['ns', 'n', 'vn', 'v'].
                    if the POS of w is not in this list, it will be filtered.
        - withFlag: if True, return a list of pair(word, weight) like posseg.cut
                    if False, return a list of words



In [61]:
print("  ".join(analyse.textrank(content, topK=20, withWeight=False, allowPOS=('ns', 'n', 'vn', 'v'))))

汽车  中国  市场  新能源  车辆  品牌  技术  发展  企业  产品  车型  服务  消费者  公司  用户  进行  系统  设计  平台  驾驶


# 用sklearn做tf-idf

In [62]:
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer(smooth_idf=False)

In [71]:
corpus = ['This is the first document.','This is the second second document.','And the third one.','Is this the first document?']

In [75]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)

In [80]:
help(vectorizer.get_feature_names)

Help on method get_feature_names in module sklearn.feature_extraction.text:

get_feature_names() method of sklearn.feature_extraction.text.TfidfVectorizer instance
    Array mapping from feature integer indices to feature name



In [83]:
vectorizer.get_feature_names()

['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']

In [78]:
X.toarray()

array([[ 0.        ,  0.43877674,  0.54197657,  0.43877674,  0.        ,
         0.        ,  0.35872874,  0.        ,  0.43877674],
       [ 0.        ,  0.27230147,  0.        ,  0.27230147,  0.        ,
         0.85322574,  0.22262429,  0.        ,  0.27230147],
       [ 0.55280532,  0.        ,  0.        ,  0.        ,  0.55280532,
         0.        ,  0.28847675,  0.55280532,  0.        ],
       [ 0.        ,  0.43877674,  0.54197657,  0.43877674,  0.        ,
         0.        ,  0.35872874,  0.        ,  0.43877674]])

# word2vec来做

In [105]:
from gensim.models.word2vec import Word2Vec

D:\ANACONDA\lib\site-packages\gensim\utils.py:865: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.


In [114]:
stopwords = list(stopwords['stopword'])

In [174]:
data = []
for line in lines:
    buff_1 = [w for w in list(jieba.cut(line)) if w not in stopwords]
    buff_2 = [w for w in buff_1 if len(w) > 1]
    data.append(buff_2)

In [175]:
data[0]

['本报',
 '济南',
 '日电',
 '潘俊强',
 '日前',
 '备受',
 '关注',
 '济南',
 '专车',
 '一审',
 '宣判',
 '济南市',
 '市中区',
 '人民法院',
 '济南',
 '市民',
 '陈超诉',
 '济南市',
 '城市',
 '公共',
 '客运',
 '管理',
 '服务中心',
 '行政处罚',
 '一案',
 '作出',
 '一审',
 '宣判',
 '判决',
 '撤销',
 '济南市',
 '客管',
 '中心',
 '专车',
 '司机',
 '陈超',
 '行政处罚']

In [124]:
# 设定词向量训练的参数
num_features = 300    # Word vector dimensionality
min_word_count = 5   # Minimum word count
num_workers = 1       # Number of threads to run in parallel
context = 5          # Context window size
downsampling = 1e-3   # Downsample setting for frequent words

In [125]:
print('Training model...')
model = Word2Vec(data, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)


Training model...


In [132]:
model.most_similar("大众")

[('一汽', 0.9911147356033325),
 ('奥迪', 0.9842259287834167),
 ('上汽', 0.9678931832313538),
 ('广汽', 0.9588107466697693),
 ('旗下', 0.9486117362976074),
 ('通用汽车', 0.9350533485412598),
 ('福特', 0.9346243143081665),
 ('吉利', 0.9339005947113037),
 ('丰田', 0.9322910308837891),
 ('大通', 0.9235988259315491)]

# LDA主题模型

In [133]:
from gensim import corpora, models, similarities
import gensim

In [158]:
stopwords=pd.read_csv("data/stopwords.txt",index_col=False,quoting=3,sep="\t",names=['stopword'], encoding='utf-8')
stopwords=stopwords['stopword'].values

In [189]:
df = pd.read_csv("./data/car_news.csv", encoding='utf-8')
df = df.dropna()
lines=df.content.values.tolist()

sentences=[]
for line in lines:
    try:
        segs=jieba.lcut(line)
        segs = list(filter(lambda x:len(x)>1, segs))
        segs = list(filter(lambda x:x not in stopwords, segs))
        sentences.append(segs)
    except Exception:
        print(line)
        continue

In [176]:
sentences = data  #这里为什么用上面的算出来是空集呢？用上上面的data就正确了

In [192]:
for word in sentences[5]:
    print(word)

编者按
新年伊始
一批
汽车
新政
如期而至
展望
2017
中国
汽车
市场
汽车
新政
无疑
汽车
市场
格局
产业
结构调整
忽视
导向
作用
影响
消费者
日常
汽车
生活


In [191]:
dictionary = corpora.Dictionary(sentences)
corpus = [dictionary.doc2bow(sentence) for sentence in sentences]

In [193]:
corpus[1]

[(7, 2),
 (10, 2),
 (20, 1),
 (24, 1),
 (25, 2),
 (26, 2),
 (28, 2),
 (29, 1),
 (30, 1),
 (31, 1),
 (32, 1),
 (33, 1),
 (34, 1),
 (35, 1),
 (36, 1),
 (37, 1),
 (38, 1),
 (39, 1),
 (40, 1),
 (41, 1),
 (42, 1)]

# LDA建模

In [181]:
lda = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=20)

In [182]:
for topic in lda.print_topics(num_topics=20, num_words=8):
    print(topic[1])

0.015*"汽车" + 0.012*"品牌" + 0.012*"中国" + 0.010*"车型" + 0.009*"产品" + 0.009*"SUV" + 0.007*"系统" + 0.007*"市场"
0.027*"中国" + 0.021*"汽车" + 0.016*"召回" + 0.010*"进口" + 0.009*"宝马" + 0.008*"品牌" + 0.008*"碳纤维" + 0.008*"指数"
0.024*"汽车" + 0.016*"品牌" + 0.012*"中国" + 0.011*"上汽" + 0.009*"市场" + 0.009*"福特" + 0.008*"中新网" + 0.007*"日电"
0.026*"设计" + 0.010*"全新" + 0.010*"动力" + 0.009*"系统" + 0.008*"车身" + 0.008*"车型" + 0.007*"采用" + 0.007*"性能"
0.010*"活动" + 0.010*"停车" + 0.008*"租赁" + 0.008*"项目" + 0.008*"停车位" + 0.007*"汽车" + 0.006*"亿元" + 0.006*"全国"
0.027*"奥迪" + 0.017*"经销商" + 0.016*"销售" + 0.016*"汽车" + 0.010*"万元" + 0.010*"市场" + 0.010*"公司" + 0.009*"优惠"
0.012*"空间" + 0.009*"电池" + 0.009*"补贴" + 0.007*"充电" + 0.007*"速度" + 0.007*"车型" + 0.007*"带来" + 0.006*"挑战"
0.075*"汽车" + 0.019*"发展" + 0.016*"中国" + 0.016*"企业" + 0.015*"市场" + 0.012*"新能源" + 0.010*"全球" + 0.009*"技术"
0.014*"汽车" + 0.010*"政策" + 0.010*"相关" + 0.008*"车辆" + 0.008*"国家" + 0.008*"建设" + 0.007*"解决方案" + 0.006*"机动车"
0.026*"服务" + 0.022*"网约车" + 0.014*"经营" + 0.011*"出租汽车" + 0.011*"平台" + 0.010